In [10]:
!wget https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt -O freq_words.txt


--2025-07-24 09:38:37--  https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75153 (73K) [text/plain]
Saving to: ‘freq_words.txt’

freq_words.txt      100%[===================>]  73.39K  --.-KB/s    in 0.02s   

2025-07-24 09:38:37 (3.39 MB/s) - ‘freq_words.txt’ saved [75153/75153]



In [11]:
braille_map = {'D': '1', 'W': '2', 'Q': '3', 'K': '4', 'O': '5', 'P': '6'}

braille_letter_map = {
    'a': '1',    'b': '12',   'c': '14',   'd': '145',  'e': '15',
    'f': '124',  'g': '1245', 'h': '125',  'i': '24',   'j': '245',
    'k': '13',   'l': '123',  'm': '134',  'n': '1345', 'o': '135',
    'p': '1234', 'q': '12345','r': '1235', 's': '234',  't': '2345',
    'u': '136',  'v': '1236', 'w': '2456', 'x': '1346', 'y': '13456','z': '1356'
}

def normalize_combo(keys):
    return ''.join(sorted([braille_map.get(k.upper(), '') for k in keys if k.upper() in braille_map]))

def word_to_braille(word):
    return [braille_letter_map.get(c, '') for c in word if c in braille_letter_map]


In [12]:
with open("freq_words.txt") as f:
    freq_words = [line.strip().lower() for line in f if line.strip().isalpha()]

word_ranks = {word: rank+1 for rank, word in enumerate(freq_words)}

braille_dict = {
    word: word_to_braille(word)
    for word in freq_words
    if all(c in braille_letter_map for c in word)
}
print(f"Loaded {len(braille_dict)} words.")


Loaded 9894 words.


In [13]:
def levenshtein_list(a, b):
    if len(a) < len(b):
        return levenshtein_list(b, a)
    if len(b) == 0:
        return len(a)
    prev = list(range(len(b)+1))
    for i, c1 in enumerate(a):
        curr = [i+1]
        for j, c2 in enumerate(b):
            ins = prev[j+1] + 1
            del_ = curr[j] + 1
            sub = prev[j] + (c1 != c2)
            curr.append(min(ins, del_, sub))
        prev = curr
    return prev[-1]

def suggest_top_k(dot_list, k=3):
    results = []
    input_len = len(dot_list)
    input_first = dot_list[0] if dot_list else ''

    for word, b_chars in braille_dict.items():
        if not b_chars:
            continue
        if abs(len(b_chars) - input_len) > 1:
            continue

        dist = levenshtein_list(dot_list, b_chars)
        max_len = max(len(dot_list), len(b_chars))
        confidence = 1.0 if dist == 0 else (1 - dist / max(len(dot_list), len(b_chars)))


        starts_same = (b_chars[0] == input_first)
        len_match_bonus = 0.1 if len(b_chars) == input_len else 0
        freq_rank = word_ranks.get(word, 10001)

        score = (
            confidence
            + (0.2 if starts_same else 0)
            + len_match_bonus
            - (freq_rank / 100000)
        )

        results.append((word, dist, freq_rank, confidence, score))

    results.sort(key=lambda x: -x[4])
    return results[:k]


In [14]:
print("Type each Braille letter as space-separated keys (e.g. D K or D Q O)")
print("Separate Braille letters by pressing Enter after each.")
print("Press Enter on an empty line to exit.\n")

while True:
    raw = input("⠿ Enter Braille word (one letter per line, empty line = exit): ").strip()
    if not raw:
        print("Exiting. Thank you!")
        break

    combos = raw.strip().split('   ')  # 3 spaces between letters
    dot_list = [normalize_combo(combo.split()) for combo in combos]

    suggestions = suggest_top_k(dot_list)

    if suggestions:
        print("Top Suggestions:")
        for word, dist, rank, confidence, score in suggestions:
            conf_percent = int(confidence * 100)
            print(f"   {word} (confidence: {conf_percent}%)")
    else:
        print("No suggestions found.")


Type each Braille letter as space-separated keys (e.g. D K or D Q O)
Separate Braille letters by pressing Enter after each.
Press Enter on an empty line to exit.

⠿ Enter Braille word (one letter per line, empty line = exit): D K   D Q O   D K O   D W
Top Suggestions:
   code (confidence: 75%)
   cod (confidence: 75%)
   cost (confidence: 50%)
⠿ Enter Braille word (one letter per line, empty line = exit): D K O   D   W Q K O   D
Top Suggestions:
   data (confidence: 100%)
   date (confidence: 75%)
   dana (confidence: 75%)
⠿ Enter Braille word (one letter per line, empty line = exit): 
Exiting. Thank you!
